You might need to install this on your system:

apt-get install python3-opencv git

In [ ]:
import os
orig_data_dir = os.getcwd()+'/images'
train_dir = os.getcwd()+"/TRAIN"
test_dir = os.getcwd()+"/TEST"
txt_file_train = os.getcwd()+'/annotations/trainval.txt'
txt_file_test = os.getcwd()+'/annotations/test.txt'

In [ ]:
if not os.path.isdir(train_dir):
  !wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz

In [ ]:
if not os.path.isdir(train_dir):
  !tar -xvf images.tar.gz

In [ ]:
!wc -l jp.txt

wc: jp.txt: No such file or directory


In [ ]:
if not os.path.isdir(train_dir):
  !wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
if not os.path.isdir(train_dir):
  !tar -xvf annotations.tar.gz

In [ ]:
if not os.path.isdir('k'):
 !git clone https://github.com/joaopauloschuler/k-neural-api.git k
else:
 !cd k && git pull

!cd k && pip install .

In [ ]:
import cai.layers
import cai.datasets
import cai.models
import cai.efficientnet
import numpy as np
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import layers
import gc
import multiprocessing
import random
import tensorflow as tf
print("Tensorflow version:", tf.version.VERSION)
print("Keras version:", keras.__version__)
import PIL
import cv2
#print("Pillow version:", PIL.PILLOW_VERSION)
from sklearn.metrics import classification_report
import shutil

Tensorflow version: 2.4.1
Keras version: 2.4.0


In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [ ]:
def CopyImagesToFolders(orig_data_dir, dest_data_dir, txt_file, verbose=True, prefix='class_'):
  # Load the CSV file.
  # Image CLASS-ID SPECIES BREED ID
  if verbose: print('Loading', txt_file)
  with open (txt_file, "r") as myfile:
    csv_lines=myfile.readlines()

  # Transform the CSV file into an array  
  line_count = 0
  a_lines = []
  for str_line in csv_lines:
    if line_count > 0: 
      a_line = str_line.replace('\n', '').split(' ')
      a_lines.append(a_line)
    line_count = line_count + 1
  
  # create destination folder
  if not os.path.isdir(dest_data_dir):
    os.mkdir(dest_data_dir)

  # Save cropped images
  line_count = 0
  failed_count = 0
  original_count = 0
  last_image_file = ''
  for a_line in a_lines:
    dest_folder_name = dest_data_dir+'/'+prefix+a_line[1]
    dest_file_name = dest_folder_name+'/'+a_line[0]
    orig_file_name = orig_data_dir+'/'+ a_line[0]
    found = False
    if os.path.isfile(orig_file_name+'.jpg'):
      dest_file_name = dest_folder_name+'/'+a_line[0]+'.jpg'
      orig_file_name = orig_data_dir+'/'+ a_line[0]+'.jpg'
      found = True
    if os.path.isfile(orig_file_name+'.png'):
      dest_file_name = dest_folder_name+'/'+a_line[0]+'.png'
      orig_file_name = orig_data_dir+'/'+ a_line[0]+'.png'
      found = True
    if verbose: print(line_count,':', a_line, dest_file_name)
    if (found):
      if not os.path.isdir(dest_folder_name):
        os.mkdir(dest_folder_name)
      if verbose: print('Copying file from', orig_file_name,'to',dest_file_name)
      shutil.copyfile(orig_file_name, dest_file_name)
    else:
      if verbose: print('File does not exist:', orig_file_name)
      failed_count = failed_count + 1
    line_count = line_count + 1
  print("Processed files:", line_count, "Failed count: ", failed_count, "Original count:", original_count)

In [ ]:
if not os.path.isdir(train_dir):
  CopyImagesToFolders(orig_data_dir=orig_data_dir, dest_data_dir=train_dir, txt_file=txt_file_train, verbose=True, prefix='class_')

In [ ]:
if not os.path.isdir(test_dir):
  CopyImagesToFolders(orig_data_dir=orig_data_dir, dest_data_dir=test_dir, txt_file=txt_file_test, verbose=True, prefix='class_')

In [ ]:
print(os.listdir(train_dir))
print(os.listdir(test_dir))

['class_4', 'class_8', 'class_12', 'class_16', 'class_20', 'class_24', 'class_28', 'class_32', 'class_36', 'class_1', 'class_5', 'class_9', 'class_13', 'class_17', 'class_21', 'class_25', 'class_29', 'class_33', 'class_37', 'class_2', 'class_6', 'class_10', 'class_14', 'class_18', 'class_22', 'class_26', 'class_30', 'class_34', 'class_3', 'class_7', 'class_11', 'class_15', 'class_19', 'class_23', 'class_27', 'class_31', 'class_35']
['class_2', 'class_6', 'class_10', 'class_14', 'class_18', 'class_22', 'class_26', 'class_30', 'class_34', 'class_27', 'class_31', 'class_3', 'class_7', 'class_11', 'class_15', 'class_19', 'class_23', 'class_4', 'class_8', 'class_12', 'class_16', 'class_20', 'class_24', 'class_28', 'class_32', 'class_35', 'class_36', 'class_1', 'class_5', 'class_9', 'class_13', 'class_17', 'class_21', 'class_25', 'class_29', 'class_33', 'class_37']


In [ ]:
datagen = cai.util.create_image_generator(rotation_range=20, width_shift_range=0.3, height_shift_range=0.3, 
  channel_shift_range=0.1) 

def lrscheduler(epoch):
  return 0.01 * (0.97**epoch)

def cyclical_adv_lrscheduler25(epoch):
    """CAI Cyclical and Advanced Learning Rate Scheduler.
    # Arguments
        epoch: integer with current epoch count.
    # Returns
        float with desired learning rate.
    """
    base_learning = 0.001
    local_epoch = epoch % 25
    if local_epoch < 7:
       return base_learning * (1 + 0.5*local_epoch)
    else:
       return (base_learning * 4) * ( 0.85**(local_epoch-7) )

# Load the dataset again with LAB

In [ ]:
num_classes = 37
batch_size = 32
epochs = 150
target_size_x = 224 # default value is 224
target_size_y = 224 # default value is 224

In [ ]:
train_x, val_x, aux_test_x, train_y, val_y, aux_test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7,
  root_dir=train_dir, lab=False, 
  verbose=True, bipolar=False, base_model_name='oxford_pets',
  has_training=True, has_validation=True, has_testing=False,
  training_size=0.9, validation_size=0.1, test_size=0.0,
  smart_resize=True,
  target_size=(target_size_x, target_size_y))

aux_train_x, aux_val_x, test_x, aux_train_y, aux_val_y, test_y, classweight, classes = cai.datasets.load_images_from_folders(seed=7,
  root_dir=test_dir, lab=False, 
  verbose=True, bipolar=False, base_model_name='oxford_pets',
  has_training=False, has_validation=False, has_testing=True,
  smart_resize=True,
  training_size=0.0, validation_size=0.0, test_size=1,
  target_size=(target_size_x, target_size_y))

print(train_x.shape,val_x.shape,test_x.shape)
print(train_y.shape,val_y.shape,test_y.shape)

Loading  37  classes.
smart resize is enabled.
loading train images
train shape is: (3309, 224, 224, 3)
loading validation images
validation shape is: (370, 224, 224, 3)
Channel  0  min: 0.0  max: 1.0
Channel  1  min: 0.0  max: 1.0
Channel  2  min: 0.0  max: 1.0
Loaded.
Loading  37  classes.
smart resize is enabled.
loading test images


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36], y=[ 0  0  0 ... 36 36 36] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


test shape is: (3668, 224, 224, 3)
Loaded.
(3309, 224, 224, 3) (370, 224, 224, 3) (3668, 224, 224, 3)
(3309, 37) (370, 37) (3668, 37)


In [ ]:
def work_on_efficientnet(show_model=False, run_fit=False, test_results=False, calc_f1=False):
  monitor='val_accuracy'
  if (calc_f1): 
    test_results=True
  if (show_model):
    input_shape = (target_size_x, target_size_y, 3)
  else:
    input_shape = (None, None, 3)
  for kType in [-1, 1, 2]:
      basefilename = 'DEV-EfficientNet-OxfordIIIPetDataset-'+str(kType)
      best_result_file_name = basefilename+'-best_result.hdf5'
      print('Running: '+basefilename)
      if kType == -1:
        model = cai.efficientnet.EfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes)
      else:
        model = cai.efficientnet.kEfficientNetB0(
          include_top=True,
          input_shape=input_shape,
          classes=num_classes,
          kType=kType)
      
      optimizer = keras.optimizers.RMSprop()
      optimizer = mixed_precision.LossScaleOptimizer(optimizer)
      model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy'])

      if (show_model): model.summary()

      save_best = keras.callbacks.ModelCheckpoint(
            filepath=best_result_file_name,
            monitor=monitor,
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            save_freq='epoch')

      if (run_fit):
        history = model.fit(
          x = datagen.flow(train_x, train_y, batch_size=batch_size),
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(val_x, val_y),
          callbacks=[save_best, tf.keras.callbacks.LearningRateScheduler(cyclical_adv_lrscheduler25)],
          workers=max([multiprocessing.cpu_count(), 4])
          )
      if (test_results):
        print('Best Model Results: '+basefilename)
        model = cai.models.load_kereas_model(best_result_file_name)
        evaluated = model.evaluate(test_x,test_y)
        for metric, name in zip(evaluated,["loss","acc"]):
              print(name,metric)
      if (calc_f1):
        pred_y = model.predict(test_x)
        print("Predicted Shape:", pred_y.shape)
        pred_classes_y = np.array(list(np.argmax(pred_y, axis=1)))
        test_classes_y = np.array(list(np.argmax(test_y, axis=1)))
        print("Pred classes shape:",pred_classes_y.shape)
        print("Test classes shape:",test_classes_y.shape)
        report = classification_report(test_classes_y, pred_classes_y, digits=4)
        print(report)
      print('Finished: '+basefilename)

# Show Models

In [ ]:
work_on_efficientnet(show_model=True, run_fit=False, test_results=False)

Running: DEV-EfficientNet-OxfordIIIPetDataset--1
Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
___________________________________

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
____________________________________________________________________________________

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
____________________________________________________________________________________

# Fitting

In [ ]:
work_on_efficientnet(show_model=False, run_fit=True, test_results=True)

Running: DEV-EfficientNet-OxfordIIIPetDataset--1
Epoch 1/150
104/104 [==============================] - 32s 160ms/step - loss: 4.7672 - accuracy: 0.0210 - val_loss: 3.7051 - val_accuracy: 0.0270

Epoch 00001: val_accuracy improved from -inf to 0.02703, saving model to DEV-EfficientNet-OxfordIIIPetDataset--1-best_result.hdf5
Epoch 2/150
104/104 [==============================] - 15s 142ms/step - loss: 3.8301 - accuracy: 0.0349 - val_loss: 8.3097 - val_accuracy: 0.0270

Epoch 00002: val_accuracy did not improve from 0.02703
Epoch 3/150
104/104 [==============================] - 16s 144ms/step - loss: 3.6671 - accuracy: 0.0434 - val_loss: 43.6792 - val_accuracy: 0.0270

Epoch 00003: val_accuracy did not improve from 0.02703
Epoch 4/150
104/104 [==============================] - 16s 142ms/step - loss: 3.6103 - accuracy: 0.0347 - val_loss: 3.7561 - val_accuracy: 0.0351

Epoch 00004: val_accuracy improved from 0.02703 to 0.03514, saving model to DEV-EfficientNet-OxfordIIIPetDataset--1-best_r

Epoch 37/150
104/104 [==============================] - 16s 142ms/step - loss: 2.9588 - accuracy: 0.1457 - val_loss: 3.0755 - val_accuracy: 0.1405

Epoch 00037: val_accuracy improved from 0.11351 to 0.14054, saving model to DEV-EfficientNet-OxfordIIIPetDataset--1-best_result.hdf5
Epoch 38/150
104/104 [==============================] - 15s 142ms/step - loss: 2.9082 - accuracy: 0.1530 - val_loss: 3.0787 - val_accuracy: 0.1486

Epoch 00038: val_accuracy improved from 0.14054 to 0.14865, saving model to DEV-EfficientNet-OxfordIIIPetDataset--1-best_result.hdf5
Epoch 39/150
104/104 [==============================] - 15s 141ms/step - loss: 2.8202 - accuracy: 0.1735 - val_loss: 3.1186 - val_accuracy: 0.1297

Epoch 00039: val_accuracy did not improve from 0.14865
Epoch 40/150
104/104 [==============================] - 16s 142ms/step - loss: 2.7960 - accuracy: 0.1814 - val_loss: 2.9813 - val_accuracy: 0.1405

Epoch 00040: val_accuracy did not improve from 0.14865
Epoch 41/150
104/104 [==========

104/104 [==============================] - 16s 144ms/step - loss: 1.7832 - accuracy: 0.4307 - val_loss: 2.2304 - val_accuracy: 0.3514

Epoch 00072: val_accuracy did not improve from 0.37297
Epoch 73/150
104/104 [==============================] - 16s 142ms/step - loss: 1.7632 - accuracy: 0.4388 - val_loss: 2.2092 - val_accuracy: 0.3595

Epoch 00073: val_accuracy did not improve from 0.37297
Epoch 74/150
104/104 [==============================] - 16s 144ms/step - loss: 1.7215 - accuracy: 0.4569 - val_loss: 2.2030 - val_accuracy: 0.3595

Epoch 00074: val_accuracy did not improve from 0.37297
Epoch 75/150
104/104 [==============================] - 15s 142ms/step - loss: 1.6881 - accuracy: 0.4578 - val_loss: 2.2359 - val_accuracy: 0.3514

Epoch 00075: val_accuracy did not improve from 0.37297
Epoch 76/150
104/104 [==============================] - 16s 142ms/step - loss: 1.8249 - accuracy: 0.4417 - val_loss: 2.1422 - val_accuracy: 0.3757

Epoch 00076: val_accuracy improved from 0.37297 to 0.

104/104 [==============================] - 16s 142ms/step - loss: 1.5047 - accuracy: 0.5276 - val_loss: 1.9968 - val_accuracy: 0.4595

Epoch 00110: val_accuracy did not improve from 0.48919
Epoch 111/150
104/104 [==============================] - 16s 143ms/step - loss: 1.4066 - accuracy: 0.5379 - val_loss: 2.4453 - val_accuracy: 0.4027

Epoch 00111: val_accuracy did not improve from 0.48919
Epoch 112/150
104/104 [==============================] - 16s 144ms/step - loss: 1.3896 - accuracy: 0.5597 - val_loss: 1.9180 - val_accuracy: 0.4405

Epoch 00112: val_accuracy did not improve from 0.48919
Epoch 113/150
104/104 [==============================] - 16s 142ms/step - loss: 1.2364 - accuracy: 0.5985 - val_loss: 1.6435 - val_accuracy: 0.5054

Epoch 00113: val_accuracy improved from 0.48919 to 0.50541, saving model to DEV-EfficientNet-OxfordIIIPetDataset--1-best_result.hdf5
Epoch 114/150
104/104 [==============================] - 15s 142ms/step - loss: 1.2426 - accuracy: 0.6074 - val_loss: 1.

104/104 [==============================] - 15s 142ms/step - loss: 0.7388 - accuracy: 0.7584 - val_loss: 1.3740 - val_accuracy: 0.6054

Epoch 00147: val_accuracy did not improve from 0.60541
Epoch 148/150
104/104 [==============================] - 15s 142ms/step - loss: 0.6849 - accuracy: 0.7772 - val_loss: 1.4884 - val_accuracy: 0.5892

Epoch 00148: val_accuracy did not improve from 0.60541
Epoch 149/150
104/104 [==============================] - 16s 143ms/step - loss: 0.6487 - accuracy: 0.7955 - val_loss: 1.4407 - val_accuracy: 0.6027

Epoch 00149: val_accuracy did not improve from 0.60541
Epoch 150/150
104/104 [==============================] - 16s 145ms/step - loss: 0.6739 - accuracy: 0.7772 - val_loss: 1.4227 - val_accuracy: 0.6216

Epoch 00150: val_accuracy improved from 0.60541 to 0.62162, saving model to DEV-EfficientNet-OxfordIIIPetDataset--1-best_result.hdf5
Best Model Results: DEV-EfficientNet-OxfordIIIPetDataset--1
115/115 [==============================] - 5s 36ms/step - lo

...


Epoch 00143: val_accuracy improved from 0.71892 to 0.72703, saving model to DEV-EfficientNet-OxfordIIIPetDataset-1-best_result.hdf5
Epoch 144/150
104/104 [==============================] - 41s 385ms/step - loss: 0.2372 - accuracy: 0.9328 - val_loss: 1.2610 - val_accuracy: 0.7081

Epoch 00144: val_accuracy did not improve from 0.72703
Epoch 145/150
104/104 [==============================] - 41s 385ms/step - loss: 0.2618 - accuracy: 0.9180 - val_loss: 1.4198 - val_accuracy: 0.7054

Epoch 00145: val_accuracy did not improve from 0.72703
Epoch 146/150
104/104 [==============================] - 41s 384ms/step - loss: 0.2285 - accuracy: 0.9246 - val_loss: 1.3769 - val_accuracy: 0.7108

Epoch 00146: val_accuracy did not improve from 0.72703
Epoch 147/150
104/104 [==============================] - 40s 382ms/step - loss: 0.2149 - accuracy: 0.9315 - val_loss: 1.4680 - val_accuracy: 0.6865

Epoch 00147: val_accuracy did not improve from 0.72703
Epoch 148/150
104/104 [============================

104/104 [==============================] - 43s 408ms/step - loss: 2.8794 - accuracy: 0.1811 - val_loss: 3.0582 - val_accuracy: 0.1676

Epoch 00027: val_accuracy did not improve from 0.21622
Epoch 28/150
104/104 [==============================] - 43s 409ms/step - loss: 2.8887 - accuracy: 0.1734 - val_loss: 3.4342 - val_accuracy: 0.1270

Epoch 00028: val_accuracy did not improve from 0.21622
Epoch 29/150
104/104 [==============================] - 43s 406ms/step - loss: 2.9514 - accuracy: 0.1795 - val_loss: 3.0449 - val_accuracy: 0.1757

Epoch 00029: val_accuracy did not improve from 0.21622
Epoch 30/150
104/104 [==============================] - 43s 405ms/step - loss: 2.9303 - accuracy: 0.1834 - val_loss: 3.5917 - val_accuracy: 0.1541

Epoch 00030: val_accuracy did not improve from 0.21622
Epoch 31/150
104/104 [==============================] - 43s 405ms/step - loss: 2.9028 - accuracy: 0.1914 - val_loss: 3.6917 - val_accuracy: 0.1459

Epoch 00031: val_accuracy did not improve from 0.2162

104/104 [==============================] - 43s 408ms/step - loss: 1.2884 - accuracy: 0.5952 - val_loss: 1.6702 - val_accuracy: 0.5649

Epoch 00063: val_accuracy improved from 0.50811 to 0.56486, saving model to DEV-EfficientNet-OxfordIIIPetDataset-2-best_result.hdf5
Epoch 64/150
104/104 [==============================] - 43s 406ms/step - loss: 1.1762 - accuracy: 0.6208 - val_loss: 1.5535 - val_accuracy: 0.5757

Epoch 00064: val_accuracy improved from 0.56486 to 0.57568, saving model to DEV-EfficientNet-OxfordIIIPetDataset-2-best_result.hdf5
Epoch 65/150
104/104 [==============================] - 43s 406ms/step - loss: 1.0933 - accuracy: 0.6631 - val_loss: 1.6149 - val_accuracy: 0.5324

Epoch 00065: val_accuracy did not improve from 0.57568
Epoch 66/150
104/104 [==============================] - 43s 406ms/step - loss: 1.0612 - accuracy: 0.6695 - val_loss: 1.5489 - val_accuracy: 0.5811

Epoch 00066: val_accuracy improved from 0.57568 to 0.58108, saving model to DEV-EfficientNet-OxfordIII

104/104 [==============================] - 43s 405ms/step - loss: 0.4385 - accuracy: 0.8562 - val_loss: 1.3523 - val_accuracy: 0.6703

Epoch 00100: val_accuracy improved from 0.66757 to 0.67027, saving model to DEV-EfficientNet-OxfordIIIPetDataset-2-best_result.hdf5
Epoch 101/150
104/104 [==============================] - 43s 406ms/step - loss: 0.5411 - accuracy: 0.8208 - val_loss: 1.3837 - val_accuracy: 0.6405

Epoch 00101: val_accuracy did not improve from 0.67027
Epoch 102/150
104/104 [==============================] - 43s 406ms/step - loss: 0.5770 - accuracy: 0.8031 - val_loss: 1.5753 - val_accuracy: 0.6324

Epoch 00102: val_accuracy did not improve from 0.67027
Epoch 103/150
104/104 [==============================] - 43s 407ms/step - loss: 0.6488 - accuracy: 0.7921 - val_loss: 1.9678 - val_accuracy: 0.5649

Epoch 00103: val_accuracy did not improve from 0.67027
Epoch 104/150
104/104 [==============================] - 43s 408ms/step - loss: 0.7737 - accuracy: 0.7530 - val_loss: 2.1

104/104 [==============================] - 43s 407ms/step - loss: 0.4944 - accuracy: 0.8370 - val_loss: 1.4047 - val_accuracy: 0.6703

Epoch 00138: val_accuracy did not improve from 0.69730
Epoch 139/150
104/104 [==============================] - 43s 408ms/step - loss: 0.4316 - accuracy: 0.8564 - val_loss: 1.7477 - val_accuracy: 0.6216

Epoch 00139: val_accuracy did not improve from 0.69730
Epoch 140/150
104/104 [==============================] - 43s 407ms/step - loss: 0.4033 - accuracy: 0.8563 - val_loss: 1.6435 - val_accuracy: 0.6135

Epoch 00140: val_accuracy did not improve from 0.69730
Epoch 141/150
104/104 [==============================] - 43s 406ms/step - loss: 0.3727 - accuracy: 0.8861 - val_loss: 1.5582 - val_accuracy: 0.6459

Epoch 00141: val_accuracy did not improve from 0.69730
Epoch 142/150
104/104 [==============================] - 43s 405ms/step - loss: 0.3362 - accuracy: 0.8871 - val_loss: 1.6469 - val_accuracy: 0.6703

Epoch 00142: val_accuracy did not improve from 0.

# Test Results

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True)

Running: DEV-EfficientNet-OxfordIIIPetDataset--1
Best Model Results: DEV-EfficientNet-OxfordIIIPetDataset--1
115/115 [==============================] - 5s 35ms/step - loss: 1.6638 - accuracy: 0.5673
loss 1.6637866497039795
acc 0.5673391222953796
Finished: DEV-EfficientNet-OxfordIIIPetDataset--1
Running: DEV-EfficientNet-OxfordIIIPetDataset-1
Best Model Results: DEV-EfficientNet-OxfordIIIPetDataset-1
115/115 [==============================] - 18s 69ms/step - loss: 1.9277 - accuracy: 0.6159
loss 1.927720546722412
acc 0.6158669590950012
Finished: DEV-EfficientNet-OxfordIIIPetDataset-1
Running: DEV-EfficientNet-OxfordIIIPetDataset-2
Best Model Results: DEV-EfficientNet-OxfordIIIPetDataset-2
115/115 [==============================] - 19s 73ms/step - loss: 1.8412 - accuracy: 0.6009
loss 1.8411823511123657
acc 0.6008723974227905
Finished: DEV-EfficientNet-OxfordIIIPetDataset-2


# Calculate F1

In [ ]:
work_on_efficientnet(show_model=False, run_fit=False, test_results=True, calc_f1=True)

Running: DEV-EfficientNet-OxfordIIIPetDataset--1
Best Model Results: DEV-EfficientNet-OxfordIIIPetDataset--1
115/115 [==============================] - 5s 35ms/step - loss: 1.6638 - accuracy: 0.5673
loss 1.6637866497039795
acc 0.5673391222953796
Predicted Shape: (3668, 37)
Pred classes shape: (3668,)
Test classes shape: (3668,)
              precision    recall  f1-score   support

           0     0.5347    0.5567    0.5455        97
           1     0.5246    0.6400    0.5766       100
           2     0.3913    0.0900    0.1463       100
           3     0.7528    0.6907    0.7204        97
           4     0.3178    0.3400    0.3285       100
           5     0.4146    0.3400    0.3736       100
           6     0.4061    0.6700    0.5057       100
           7     0.6667    0.7000    0.6829       100
           8     0.5841    0.6600    0.6197       100
           9     0.6757    0.7500    0.7109       100
          10     0.7043    0.8182    0.7570        99
          11     0.50